# Data Loading, and initial pre-processing

This notebook was used to load the initial CSV data into Pandas, to make some initial corrections (as detailed in the report, and commented below) and also to derive some additional variables based on the rate data provided by TFL.

In [1]:
# Imports and directives
import calendar
import math
import numpy as np
import pandas as pd
import sys
from datetime import datetime

In [2]:
# Library versions, for info
print ('Developed with the following library versions:')
print ('Python:\t\t\x1b[34m%s\x1b[0m' % (sys.version))
print ('Pandas:\t\t\x1b[34m%s\x1b[0m' % (pd.__version__))

Developed with the following library versions:
Python:		2.7.11 |Anaconda 4.0.0 (64-bit)| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
Pandas:		0.18.0


In [72]:
# Date last worked on
datetime.now()

datetime.datetime(2017, 1, 19, 20, 30, 26, 737000)

In [3]:
# Top level declarations
journeys_data_file = 'D:\\cyclehire\\JourneyData2016.csv'
stations_data_file = 'D:\\cyclehire\\stations.csv'
rad = math.pi / 180
R = 6378.145

In [9]:
# Only reading the columns we care about, for simplicity
journeys_cols = [0,1,2,4,6,7]

journeys_names = [
    'RentalId',
    'Duration',
    'BikeId',
    'EndStationId',
    'StartDateTime',
    'StartStationId'
]

# Dict of data types to pass into read_csv
journeys_dtype = {
    'RentalId': int,
    'Duration': float, #to handle NAs
    'BikeId': int,
    'EndStationId': float, #to handle NAs
    'StartDateTime': int,
    'StartStationId': int
}

# Convert date strings to epoch (sort of) values.
# Note that dates are specified in the TFL data as BST, which leads 
# to some oddities e.g. a record immediately after the October
# date roll will have a smaller timestamp than one immediately 
# before it. We could try and correct for this, but really we're
# only using timestamps for efficiency (to avoid using Python
# objects) - and since we're only ever going to be interested in 
# what the local time was, it doesn't really matter. That is,
# on 30th October 2016, 01:59 really was 01:59, locally; and 
# so was 01:01 - even though the latter came after the former!
def timestampConverter(datestring):
    dateobj = datetime.strptime(datestring, '%d/%m/%Y %H:%M')
    return calendar.timegm(dateobj.utctimetuple())

# dict of converters to pass into read_csv
journeys_converters = {
    'StartDateTime' : timestampConverter
}

# Read the file
journeys = pd.read_csv(
    journeys_data_file,
    header = 0,
    names = journeys_names,
    usecols = journeys_cols, 
    converters = journeys_converters,
    dtype = journeys_dtype
)

# Sanity check
print ('shape: %s' % (str(journeys.shape)))
print ('dtypes:\n%s' % (str(journeys.dtypes)))

shape: (9454907, 6)
dtypes:
RentalId            int32
Duration          float64
BikeId              int32
EndStationId      float64
StartDateTime       int64
StartStationId      int32
dtype: object


In [13]:
# Correct duff data due to DST changes. The list below came from manual inspection of
# the raw data around the 30th March DST change (clocks going forward). It should be
# possible to do this programatically (e.g. look for journeys which begin/end either
# side of 1am on 27th March) but since there were so few of them it was quicker (and
# safer) to do it manually. 
incorrectRentalIds = [
    52440126,
    52440127,
    52440128,
    52440130,
    52440131,
    52440132,
    52440133,
    52440134,
    52440135,
    52440136,
    52440137,
    52440138,
    52440139,
    52440140,
    52440141,
    52440142
]

def correctDstDurations(journey):
    duration = journey.Duration
    if journey.RentalId in incorrectRentalIds:
        # Needs fixing due to March DST change
        duration = duration - 3600
    # Else needs fixing for October DST change if duration -ve
    return duration+3600 if duration < 0 else duration

journeys['Duration'] = journeys.apply(correctDstDurations, axis=1)

In [24]:
# Prob bit inefficient (could probably do all in one go), but..
journeys['HourOfDay'] = journeys.StartDateTime.apply(lambda x : datetime.fromtimestamp(x).hour)
journeys['MonthOfYear'] = journeys.StartDateTime.apply(lambda x : datetime.fromtimestamp(x).month)
journeys['DayOfWeek'] = journeys.StartDateTime.apply(lambda x : datetime.fromtimestamp(x).isoweekday())
journeys['DayOfYear'] = journeys.StartDateTime.apply(lambda x : int(format(datetime.fromtimestamp(x), '%j')))

In [25]:
# Using ints - otherwise takes too long to parse strings
def inferJourneyType(journey):
    if math.isnan(journey.EndStationId):
        return 0 # Unknown
    elif journey.StartStationId == journey.EndStationId:
        return 2 # Circular
    else:
        return 1 # Regular
    
journeys["JourneyType"] = journeys.apply(inferJourneyType, axis=1)

In [26]:
# Using ints - otherwise takes too long to parse strings
def inferSeason(journey):
    monthOfYear = journey.MonthOfYear
    if monthOfYear in [12,1,2]:
        return 1 # Winter
    elif monthOfYear in [3,4,5]:
        return 2 # Spring
    elif monthOfYear in [6,7,8]:
        return 3 # Summer
    else:
        return 4 # Autumn
    
journeys["Season"] = journeys.apply(inferSeason, axis=1)

In [27]:
# Read in the docking stations data. 
# Only care about latitude/longitude values.
stations_cols = [
    'id',
    'lat',
    'long'
]

stations = pd.read_csv(
    stations_data_file,
    usecols = stations_cols
)

print ('shape: %s' % (str(stations.shape)))
print ('dtypes:\n%s' % (str(stations.dtypes)))

shape: (787, 3)
dtypes:
id        int64
lat     float64
long    float64
dtype: object


In [28]:
# Add in the geo-coordinates to the journeys data
journeys = journeys.merge(
    # Add lat/long of StartStationId
    stations, 
    left_on = 'StartStationId', 
    right_on = "id",
    how = 'left' # StartStationId is never null in the journeys data, but may not be known in the stations data
).merge(
    # Add lat/long of EndStationId
    stations, 
    left_on = 'EndStationId', 
    right_on = "id",
    how = 'left' # EndStationId is sometimes null, and also may not be known
).drop(
    # We don't want the extra id columns we used for the join
    labels = ["id_x", "id_y"],
    axis = 1
).rename(
    # And we want some more meaningful column names
    columns={
        'lat_x' :'StartLat',
        'long_x':'StartLong',
        'lat_y' :'EndLat',
        'long_y':'EndLong'
    }
)

In [29]:
# Add distance metric. Note this has been adapted from an R script on the web.
# For details see https://conservationecology.wordpress.com/2013/06/30/distance-between-two-points-in-r
def haversine_distance(journey):
    a1 = journey['StartLat'] * rad
    a2 = journey['StartLong'] * rad
    b1 = journey['EndLat'] * rad
    b2 = journey['EndLong'] * rad
    dlon = b2 - a2
    dlat = b1 - a1
    a = math.pow(math.sin(dlat/2),2) + math.cos(a1) * math.cos(b1) * math.pow(math.sin(dlon/2),2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d

In [30]:
journeys['Distance'] = journeys.apply(haversine_distance, axis=1)

In [43]:
# Each journey has an end location, which may not be the same as the
# start location of the journey which follows. This would take ages to 
# derive this on the fly so I'm printing it to tape, so to speak.
def findNextStartStation(row):
    bikeid = row.BikeId
    ix = row.name
    nextix = ix+1 # Assumes journeys are sorted chronologically, and indexed sequentially.
    if nextix < journeys.shape[0]:
        nextrow = journeys.iloc[nextix]
        nextbikeid = nextrow.BikeId
        if bikeid == nextbikeid:
            return nextrow.StartStationId
    return np.NAN

# Similar logic for gap to next journey, for a given bike.
def findNextStartDateTime(row):
    bikeid = row.BikeId
    ix = row.name
    nextix = ix+1 # Assumes journeys are sorted chronologically, and indexed sequentially.
    if nextix < journeys.shape[0]:
        nextrow = journeys.iloc[nextix]
        nextbikeid = nextrow.BikeId
        if bikeid == nextbikeid:
            return int(nextrow.StartDateTime)
    return np.NAN

# Next RentalId also useful.
def findNextRental(row):
    bikeid = row.BikeId
    ix = row.name
    nextix = ix+1 # Assumes journeys are sorted chronologically, and indexed sequentially.
    if nextix < journeys.shape[0]:
        nextrow = journeys.iloc[nextix]
        nextbikeid = nextrow.BikeId
        if bikeid == nextbikeid:
            return int(nextrow.RentalId)
    return np.NAN

In [55]:
# To apply the functions above, we first need to sort by BikeId and chronologically
journeys = journeys.sort_values(by=["BikeId","StartDateTime"])
# Also we need the index to be 0,1,2,3,4 etc. 
journeys = journeys.set_index([[i for i in range(0,journeys.shape[0])]])
# Now derive the additional columns relating to the next journey after this one
journeys["NextRentalId"] = journeys.apply(findNextRental, axis=1)
journeys["NextStartStationId"] = journeys.apply(findNextStartStation, axis=1)
journeys["NextStartDateTime"] = journeys.apply(findNextStartDateTime, axis=1)
journeys["GapToNextJourney"] = journeys.NextStartDateTime - (journeys.StartDateTime + journeys.Duration)

In [70]:
# Take a peek for sanity
journeys = journeys.sort_values(by="RentalId")
journeys.head()

,RentalId,Duration,BikeId,EndStationId,StartDateTime,StartStationId,HourOfDay,MonthOfYear,DayOfWeek,DayOfYear,...,Season,StartLat,StartLong,EndLat,EndLong,Distance,NextStartStationId,NextStartDateTime,GapToNextJourney,NextRentalId
705758,50608184,4440.0,1104,21.0,1451606400,98,0,1,5,1,...,1,51.525542,-0.138231,51.530078,-0.138846,0.506753,233.0,1.451819e+09,208380.0,50627904.0
342876,50608186,1200.0,529,118.0,1451606640,419,0,1,5,1,...,1,51.485821,-0.149004,51.495827,-0.135478,1.455894,118.0,1.451609e+09,960.0,50608716.0
5345001,50608187,1200.0,8452,118.0,1451606640,419,0,1,5,1,...,1,51.485821,-0.149004,51.495827,-0.135478,1.455894,118.0,1.451609e+09,960.0,50608711.0
5649381,50608188,1080.0,8934,251.0,1451606640,66,0,1,5,1,...,1,51.517950,-0.108657,51.518908,-0.079249,2.039912,251.0,1.451629e+09,21660.0,50610937.0
8556786,50608189,1080.0,13194,251.0,1451606700,66,0,1,5,1,...,1,51.517950,-0.108657,51.518908,-0.079249,2.039912,251.0,1.451670e+09,61740.0,50616667.0


In [71]:
# Write out the data frame as we don't want to have to repeat this. 
journeys.to_csv("D:\\cyclehire\\JourneyData2016.frame.csv")